In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import dspy

# Optimized for M2 Mac
HUGGINGFACE_MODEL = 'distilgpt2'  # Lightweight model for performance
CSV_PATH = "./151_ideas_updated.csv"

# Load and clean the dataset
def load_data(csv_path):
    try:
        data = pd.read_csv(csv_path, on_bad_lines='skip')
        return data.dropna().reset_index(drop=True)
    except Exception as e:
        print(f"Error loading CSV: {e}")
        exit(1)

data = load_data(CSV_PATH)

# Load tokenizer and model with MPS (Metal Performance Shaders) support for M2
def load_model_and_tokenizer(model_name):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        ).to(device)

        text_generator = pipeline(
            'text-generation', 
            model=model, 
            tokenizer=tokenizer,
            device=0 if device.type == 'mps' else -1
        )

        return text_generator
    except Exception as e:
        print(f"Model loading error: {e}")
        exit(1)

text_generator = load_model_and_tokenizer(HUGGINGFACE_MODEL)

# Custom Language Model for DSPy
class SimpleLLM(dspy.LM):
    def __init__(self, generator):
        self.generator = generator

    def __call__(self, prompt, **kwargs):
        try:
            responses = self.generator(
                prompt, 
                max_length=100, 
                num_return_sequences=1,
                do_sample=True,
                temperature=0.7
            )
            return responses[0]['generated_text']
        except Exception as e:
            print(f"Generation error: {e}")
            return "I'm having trouble generating a response."

# Define chatbot signature
class ChatbotSignature(dspy.Signature):
    """Generate a helpful and concise response to a user's query."""
    query = dspy.InputField()
    response = dspy.OutputField(desc="Helpful and relevant answer")

# Configure DSPy
dspy.settings.configure(lm=SimpleLLM(text_generator))

# Create prediction module
chatbot = dspy.Predict(ChatbotSignature)

# Define Chain of Thought model
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    rationale = dspy.OutputField(desc="Reasoning before providing the answer")
    answer = dspy.OutputField(desc="Final concise answer")

# Initialize Chain of Thought model
chain_of_thought = dspy.ChainOfThought(BasicQA)

# Interactive chat loop
def chat():
    print("DSPy Chatbot: Hello! I'm ready to help. Type 'exit' to quit.")
    print("Dataset columns:", list(data.columns))

    while True:
        try:
            user_input = input("You: ")

            if user_input.lower() == 'exit':
                print("DSPy Chatbot: Goodbye!")
                break

            # Add context from the dataset
            context = data['Ideas'].sample(1).values[0]
            enhanced_prompt = f"Context from dataset: {context}\n\nUser query: {user_input}"

            # Generate response with Chain of Thought
            pred = chain_of_thought(question=enhanced_prompt)
            print("Thought:", pred.rationale)
            print("DSPy Chatbot:", pred.answer)
        except Exception as e:
            print(f"Chat error: {e}")

# Run the chatbot
if __name__ == "__main__":
    chat()


DSPy Chatbot: Hello! I'm ready to help. Type 'exit' to quit.
Dataset columns: ['Ideas', 'Theme a', 'Theme-b', 'Theme-c', 'Unnamed: 4', 'Unnamed: 5']


In [1]:
# chatgpt jupyter
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import dspy

# Optimized for M2 Mac
HUGGINGFACE_MODEL = 'distilgpt2'  # Lightweight model for performance
CSV_PATH = "./151_ideas_updated.csv"

# Load and clean the dataset
def load_data(csv_path):
    try:
        data = pd.read_csv(csv_path, on_bad_lines='skip')
        return data.dropna().reset_index(drop=True)
    except Exception as e:
        print(f"Error loading CSV: {e}")
        exit(1)

data = load_data(CSV_PATH)

# Load tokenizer and model with MPS (Metal Performance Shaders) support for M2
def load_model_and_tokenizer(model_name):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        ).to(device)

        text_generator = pipeline(
            'text-generation', 
            model=model, 
            tokenizer=tokenizer,
            device=0 if device.type == 'mps' else -1
        )

        return text_generator
    except Exception as e:
        print(f"Model loading error: {e}")
        exit(1)

text_generator = load_model_and_tokenizer(HUGGINGFACE_MODEL)

# Custom Language Model for DSPy
class SimpleLLM(dspy.LM):
    def __init__(self, generator):
        self.generator = generator

    def __call__(self, prompt, **kwargs):
        try:
            responses = self.generator(
                prompt, 
                max_length=100, 
                num_return_sequences=1,
                do_sample=True,
                temperature=0.7
            )
            return responses[0]['generated_text']
        except Exception as e:
            print(f"Generation error: {e}")
            return "I'm having trouble generating a response."

# Define chatbot signature
class ChatbotSignature(dspy.Signature):
    """Generate a helpful and concise response to a user's query."""
    query = dspy.InputField()
    response = dspy.OutputField(desc="Helpful and relevant answer")

# Configure DSPy
dspy.settings.configure(lm=SimpleLLM(text_generator))

# Create prediction module
chatbot = dspy.Predict(ChatbotSignature)

# Define Chain of Thought model
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    rationale = dspy.OutputField(desc="Reasoning before providing the answer")
    answer = dspy.OutputField(desc="Final concise answer")

# Initialize Chain of Thought model
chain_of_thought = dspy.ChainOfThought(BasicQA)

# Interactive chat loop for Jupyter Notebook
def chat_jupyter():
    from IPython.display import display, clear_output
    import ipywidgets as widgets

    # Display setup
    input_box = widgets.Text(description="You:", placeholder="Type your question here...")
    output_area = widgets.Output()
    send_button = widgets.Button(description="Send")
    quit_button = widgets.Button(description="Exit")

    def handle_send(_):
        with output_area:
            clear_output(wait=True)
            user_input = input_box.value.strip()
            input_box.value = ""

            if user_input.lower() == 'exit':
                print("DSPy Chatbot: Goodbye!")
                return

            try:
                # Add context from the dataset
                context = data['Ideas'].sample(1).values[0]
                enhanced_prompt = f"Context from dataset: {context}\n\nUser query: {user_input}"

                # Generate response with Chain of Thought
                pred = chain_of_thought(question=enhanced_prompt)
                print(f"You: {user_input}")
                print("Thought:", pred.rationale)
                print("DSPy Chatbot:", pred.answer)
            except Exception as e:
                print(f"Chat error: {e}")

    def handle_exit(_):
        with output_area:
            clear_output()
            print("DSPy Chatbot: Goodbye!")

    # Bind actions to buttons
    send_button.on_click(handle_send)
    quit_button.on_click(handle_exit)

    # Layout
    display(input_box, send_button, quit_button, output_area)

# Run the chatbot in Jupyter
chat_jupyter()


Text(value='', description='You:', placeholder='Type your question here...')

Button(description='Send', style=ButtonStyle())

Button(description='Exit', style=ButtonStyle())

Output()